In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf

from pyspark.sql import SparkSession
from pyspark.sql import types 
from pyspark.sql.functions import col, udf, rand, monotonically_increasing_id
from pyspark.sql.types import StringType

from pyspark.sql.functions import split, size
from pyspark.sql.functions import isnan, isnull, when, count

In [3]:
sc = SparkContext()
spark = SparkSession(sc)

## Cho dữ liệu DallasCouncilVoters.csv

### 1. Đọc dữ liệu => df

In [4]:
df = spark.read.csv("../../Data/DallasCouncilVoters.csv", header=True, inferSchema=True)

### 2. Cho biết dữ liệu có bao nhiêu dòng, in scheme. Hiển thị 5 dòng dữ liệu đầu tiên.

In [5]:
df.count()

44625

In [6]:
df.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)



In [7]:
df.show(5)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
+----------+-------------+-------------------+
only showing top 5 rows



### 3. Kiểm tra dữ liệu NaN, null. Nếu dòng nào 'VOTER_NAME' có dữ liệ null thì xóa hết các dòng đó.

In [8]:
df = df.dropna(subset='VOTER_NAME')

In [9]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,0
VOTER_NAME,0


=> Hết dữ liệu null.

### 4. Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng.

In [10]:
num_rows = df.count()
num_dist_rows = df.distinct().count()
dup_rows = num_rows - num_dist_rows

In [11]:
display(num_rows, num_dist_rows, dup_rows)

44122

1273

42849

In [12]:
# Check duplicate
df.filter(df['VOTER_NAME'] == 'Philip T. Kingston').show(5)

+----------+-------------+------------------+
|      DATE|        TITLE|        VOTER_NAME|
+----------+-------------+------------------+
|02/08/2017|Councilmember|Philip T. Kingston|
|02/08/2017|Councilmember|Philip T. Kingston|
|01/11/2017|Councilmember|Philip T. Kingston|
|09/14/2016|Councilmember|Philip T. Kingston|
|01/04/2017|Councilmember|Philip T. Kingston|
+----------+-------------+------------------+
only showing top 5 rows



In [13]:
df = df.drop_duplicates()

In [14]:
df.count()

1273

### 5. Tìm các VOTER_NAME duy nhất và hiển thị 10 dòng đầu tiên.

In [15]:
# Show the distinct VOTER_NAME entries
df.select(df['VOTER_NAME']).distinct().show(10)

+--------------------+
|          VOTER_NAME|
+--------------------+
|      Tennell Atkins|
|  the  final   20...|
|        Scott Griggs|
|       Scott  Griggs|
|       Sandy Greyson|
| Michael S. Rawlings|
| the final 2018 A...|
|        Kevin Felder|
|        Adam Medrano|
|       Casey  Thomas|
+--------------------+
only showing top 10 rows



### 6. Lọc dữ liệu theo điều kiện 'VOTER_NAME' duy nhất có chiều dài 1-20 ký tự.

In [16]:
df = df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

In [17]:
df.show(5)

+----------+--------------------+------------------+
|      DATE|               TITLE|        VOTER_NAME|
+----------+--------------------+------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|
|02/14/2018|       Councilmember|   Lee M. Kleinman|
|04/25/2018|       Councilmember|    Tennell Atkins|
|08/29/2018|       Councilmember|      Kevin Felder|
|10/18/2017|       Councilmember|Jennifer S.  Gates|
+----------+--------------------+------------------+
only showing top 5 rows



### 7. Lọc bỏ các dữ liệu mà trong 'VOTER_NAME' có chứa dấu '_' (underscore).

In [18]:
# Filter out df where the VOTER_NAME contains an underscore
df = df.filter(~ col('VOTER_NAME').contains('_'))

In [19]:
# Show the distinct VOTER_NAME entries again
df.select(df['VOTER_NAME']).distinct().show(10)

+-------------------+
|         VOTER_NAME|
+-------------------+
|     Tennell Atkins|
|       Scott Griggs|
|      Scott  Griggs|
|      Sandy Greyson|
|Michael S. Rawlings|
|       Kevin Felder|
|       Adam Medrano|
|      Casey  Thomas|
|      Mark  Clayton|
|       Casey Thomas|
+-------------------+
only showing top 10 rows



## Modifying DataFrame 

### 8. Tạo cột 'splits' chứa thông tin được cắt theo khoảng trắng từ 'VOTER_NAME'.

In [20]:
df = df.withColumn('splits', split(df.VOTER_NAME, '\s+'))

### 9. Tạo cột 'first_name' lấy dữ liệu từ phần tử đầu tiên của cột 'splits'.

In [21]:
df = df.withColumn('first_name', df.splits.getItem(0))

### 10. Tạo cột 'last_name' lấy dữ liệu từ phần tử cuối cùng của cột 'splits'.

In [22]:
df = df.withColumn('first_name', df.splits.getItem(size('splits') - 1))

In [23]:
# Show the voter_df DataFrame
df.show(3)

+----------+--------------------+---------------+-------------------+----------+
|      DATE|               TITLE|     VOTER_NAME|             splits|first_name|
+----------+--------------------+---------------+-------------------+----------+
|04/11/2018|Deputy Mayor Pro Tem|   Adam Medrano|    [Adam, Medrano]|   Medrano|
|02/14/2018|       Councilmember|Lee M. Kleinman|[Lee, M., Kleinman]|  Kleinman|
|04/25/2018|       Councilmember| Tennell Atkins|  [Tennell, Atkins]|    Atkins|
+----------+--------------------+---------------+-------------------+----------+
only showing top 3 rows



### 11. Tạo cột 'random_val' theo điều kiện: nếu cột 'TITLE' có nội dung là 'Councilmember' thì 'random_val' sẽ có giá trị rand(), nếu nội dung là 'Mayor' thì 'random_val' sẽ có giá trị 2, ngược lại sẽ có giá trị là 0.

In [24]:
# Adding a column to df for any voter with the title 'Councilmember'
df = df.withColumn('random_val', when(df.TITLE == 'Councilmember', rand()))

In [25]:
# Show some of the dataframe rows, noting whether the when clause worked 
df.show(5)

+----------+--------------------+------------------+--------------------+----------+------------------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|        random_val|
+----------+--------------------+------------------+--------------------+----------+------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|   Medrano|              null|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|  Kleinman|0.7449141937080128|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|    Atkins| 0.801191174754082|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|    Felder|0.8337675897812277|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|     Gates|0.7114314184223407|
+----------+--------------------+------------------+--------------------+----------+------------------+
only showing top 5 rows



In [26]:
# Add a column to df for a voter based on their position
df = df.withColumn('random_val', when(df.TITLE == 'Councilmember', rand())
                                .when(df.TITLE == 'Mayor', 2)
                                .otherwise(0))

In [27]:
# Show some of the dataframe rows
df.show(5)

+----------+--------------------+------------------+--------------------+----------+--------------------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|          random_val|
+----------+--------------------+------------------+--------------------+----------+--------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|   Medrano|                 0.0|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|  Kleinman| 0.09600816510206334|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|    Atkins|  0.6816190290348454|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|    Felder|  0.8091277589376191|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|     Gates|0.013637867002685766|
+----------+--------------------+------------------+--------------------+----------+--------------------+
only showing top 5 rows



### 12. Lọc các dòng dữ liệu có 'random_val'=0. Hiển thị

In [28]:
df.filter(df.random_val == 0).show(5)

+----------+--------------------+-----------------+--------------------+----------+----------+
|      DATE|               TITLE|       VOTER_NAME|              splits|first_name|random_val|
+----------+--------------------+-----------------+--------------------+----------+----------+
|04/11/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|   Medrano|       0.0|
|04/12/2017|       Mayor Pro Tem| Monica R. Alonzo|[Monica, R., Alonzo]|    Alonzo|       0.0|
|06/28/2017|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|   Medrano|       0.0|
|01/03/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|   Medrano|       0.0|
|01/17/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|   Caraway|       0.0|
+----------+--------------------+-----------------+--------------------+----------+----------+
only showing top 5 rows



### 13. Xây dựng function: getFirstAndMiddle(names) trả về kết quả gồm First và Middle (names). Khai báo function vừa viết dưới dạng udf đặt tên là udfFirstAndMiddle.

In [29]:
def getFirstAndMiddle(names):
    # Return a space separated string of names
    return ' '.join(names[:-1])

In [30]:
udfFirstAndMiddle = udf(getFirstAndMiddle, StringType())

### 14. Tạo cột first_and_middle_name bằng cách gọi udf trên với tham số truyền vào là cột 'splits'. In kết quả.

In [31]:
df = df.withColumn('first_and_middle_name', udfFirstAndMiddle(df.splits))

### 15. Xóa bỏ các cột 'first_name', 'splits'. In kết quả.

In [32]:
df = df.drop('first_name')
df = df.drop('splits')

In [33]:
df.show(5)

+----------+--------------------+------------------+--------------------+---------------------+
|      DATE|               TITLE|        VOTER_NAME|          random_val|first_and_middle_name|
+----------+--------------------+------------------+--------------------+---------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|                 0.0|                 Adam|
|02/14/2018|       Councilmember|   Lee M. Kleinman| 0.09600816510206334|               Lee M.|
|04/25/2018|       Councilmember|    Tennell Atkins|  0.6816190290348454|              Tennell|
|08/29/2018|       Councilmember|      Kevin Felder|  0.8091277589376191|                Kevin|
|10/18/2017|       Councilmember|Jennifer S.  Gates|0.013637867002685766|          Jennifer S.|
+----------+--------------------+------------------+--------------------+---------------------+
only showing top 5 rows



## Adding an ID Field

In [34]:
# Select all the unique council voters
df = df.select(df['VOTER_NAME']).distinct()
# Count the rows in voter_df
print('\n There are %d rows in the df dataframe' % df.count())


 There are 27 rows in the df dataframe


### 16. Thêm cột 'ROW_ID' bằng phương thức: monotonically_increasing_id() (trong pyspark.sql.functions).

In [35]:
df = df.withColumn('ROW_ID', monotonically_increasing_id())

### 17. Hiển thị 10 dòng đầu tiên của dữ liệu với ROW_ID tăng dần.

In [36]:
df.orderBy(df.ROW_ID.desc()).show(10)

+-------------------+-------------+
|         VOTER_NAME|       ROW_ID|
+-------------------+-------------+
|       Lee Kleinman|1709396983808|
|        Erik Wilson|1700807049216|
|Carolyn King Arnold|1632087572480|
|Rickey D.  Callahan|1597727834112|
|   Monica R. Alonzo|1382979469312|
|    Lee M. Kleinman|1228360646656|
|  Jennifer S. Gates|1194000908288|
|Philip T.  Kingston|1185410973696|
|  Dwaine R. Caraway|1142461300736|
| Rickey D. Callahan|1125281431553|
+-------------------+-------------+
only showing top 10 rows



## IDs with different partitions

In [37]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the df Dataframe.\n" % df.rdd.getNumPartitions())


There are 200 partitions in the df Dataframe.



* Make sure to store the result of .rdd.max().[0] in the variable.
* monotonically_increasing_id() return an integer. You can modify that value in-line.
* Make sure to show both Data Frames.

In [38]:
# Detemine the highest ROW_ID and save it in previous_max_id
previous_max_id = df.select('ROW_ID').rdd.max()[0]
# Add a RORW_ID column to df_april starting at the desired value
voter_df_april = df.withColumn('ROW_ID', monotonically_increasing_id() + previous_max_id)

In [39]:
# Show the ROW_ID from both DataFrames and compare
df.select('ROW_ID').show(5)
voter_df_april.select('ROW_ID').show(5)

+------------+
|      ROW_ID|
+------------+
|  8589934592|
| 34359738368|
| 42949672960|
| 51539607552|
|103079215104|
+------------+
only showing top 5 rows

+-------------+
|       ROW_ID|
+-------------+
|1717986918400|
|1743756722176|
|1752346656768|
|1760936591360|
|1812476198912|
+-------------+
only showing top 5 rows

